# 🔁 Validación Cruzada

Este notebook muestra ejemplos prácticos de validación cruzada utilizando scikit-learn:
- Holdout
- K-Fold
- Shuffle Split
- Leave-One-Out (LOOCV)

También se sugiere comparar los resultados para elegir el mejor enfoque para tus modelos.

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (
    train_test_split,
    KFold,
    ShuffleSplit,
    LeaveOneOut,
    cross_val_score
)
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# 1. Cargar y preprocesar datos
try:
    data = pd.read_csv('insurance.csv')
except FileNotFoundError:
    print("Asegúrate de que el archivo 'insurance.csv' esté en el directorio correcto.")
    # Datos de ejemplo para que el script no falle
    data = pd.DataFrame({
        'age': [19, 33, 28, 62, 46], 'sex': ['female', 'male', 'male', 'female', 'female'],
        'bmi': [27.9, 22.7, 33.0, 26.2, 23.3], 'children': [0, 1, 3, 0, 2],
        'smoker': ['yes', 'no', 'no', 'yes', 'no'], 'region': ['southwest', 'southeast', 'southeast', 'northwest', 'southwest'],
        'charges': [16884.92, 1725.55, 4449.46, 27808.72, 3450.00]
    })

X = data.drop(columns=['charges'])
y = data['charges']

# Codificar variables categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['sex', 'smoker', 'region'])
    ],
    remainder='passthrough'
)
X_processed = preprocessor.fit_transform(X)

# 2. Definir modelos
models = {
    "Linear Regression": LinearRegression(),
    "KNN": KNeighborsRegressor(n_neighbors=5)
}

# 3. Técnicas de Validación Cruzada
cv_methods = {
    "Holdout": train_test_split(X_processed, y, test_size=0.3, random_state=42),
    "KFold (5 folds)": KFold(n_splits=5, shuffle=True, random_state=42),
    "Shuffle Split": ShuffleSplit(n_splits=5, test_size=0.3, random_state=42),
    #"Leave-One-Out": LeaveOneOut()
}

# 4. Evaluación por técnica y modelo
results = {}
for cv_name, cv in cv_methods.items():
    print(f"\n🔹 {cv_name.upper()}")
    results[cv_name] = {}

    for model_name, model in models.items():
        if cv_name == "Holdout":
            X_train, X_test, y_train, y_test = cv
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            results[cv_name][model_name] = {"R²": r2, "MAE": mae}
            print(f"\n📌 {model_name}:")
            print(f"  - R²: {r2:.3f}")
            print(f"  - MAE: ${mae:.2f}")
        else:
            r2_scores = cross_val_score(model, X_processed, y, cv=cv, scoring='r2')
            mae_scores = -cross_val_score(model, X_processed, y, cv=cv, scoring='neg_mean_absolute_error')
            results[cv_name][model_name] = {
                "R²": np.mean(r2_scores),
                "MAE": np.mean(mae_scores)
            }
            print(f"\n📌 {model_name}:")
            print(f"  - R² promedio: {np.mean(r2_scores):.3f}")
            print(f"  - MAE promedio: ${np.mean(mae_scores):.2f}")

# 5. Análisis por segmentos (fumadores/no fumadores) con K-Fold (ejemplo)
print("\n🔎 ANÁLISIS POR SEGMENTOS (KFold)")
for model_name, model in models.items():
    print(f"\n📊 {model_name}:")
    for grupo in ['no', 'yes']:
        mask = data['smoker'] == grupo
        X_grupo = X[mask]
        y_grupo = y[mask]
        X_processed_grupo = preprocessor.transform(X_grupo)

        # --- CÓDIGO CORREGIDO Y MÁS ROBUSTO ---
        n_splits = 5
        # Para que cada pliegue de prueba tenga al menos 2 muestras,
        # el tamaño total del grupo debe ser al menos 2 * n_splits.
        if len(y_grupo) < 2 * n_splits:
            print(f"  🎯 { 'No fumadores' if grupo == 'no' else 'Fumadores'}:")
            print(f"     - Grupo demasiado pequeño ({len(y_grupo)} muestras) para una validación robusta con {n_splits} pliegues.")
            continue # Salta al siguiente grupo

        # Si el grupo es suficientemente grande, proceder con la validación
        kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        r2_scores = cross_val_score(model, X_processed_grupo, y_grupo, cv=kfold)
        mae_scores = -cross_val_score(model, X_processed_grupo, y_grupo, cv=kfold, scoring='neg_mean_absolute_error')

        nombre_grupo = "No fumadores" if grupo == 'no' else "Fumadores"
        print(f"  🎯 {nombre_grupo}:")
        print(f"    - R² promedio: {np.mean(r2_scores):.3f}")
        print(f"    - MAE promedio: ${np.mean(mae_scores):.2f}")


🔹 HOLDOUT

📌 Linear Regression:
  - R²: 0.982
  - MAE: $666.75

📌 KNN:
  - R²: 0.374
  - MAE: $3711.16

🔹 KFOLD (5 FOLDS)

📌 Linear Regression:
  - R² promedio: 0.963
  - MAE promedio: $844.18

📌 KNN:
  - R² promedio: 0.328
  - MAE promedio: $3684.99

🔹 SHUFFLE SPLIT

📌 Linear Regression:
  - R² promedio: 0.971
  - MAE promedio: $779.59

📌 KNN:
  - R² promedio: 0.315
  - MAE promedio: $3832.75

🔎 ANÁLISIS POR SEGMENTOS (KFold)

📊 Linear Regression:
  🎯 No fumadores:
    - R² promedio: 0.887
    - MAE promedio: $837.85
  🎯 Fumadores:
    - R² promedio: 0.815
    - MAE promedio: $1154.10

📊 KNN:
  🎯 No fumadores:
    - R² promedio: 0.790
    - MAE promedio: $1162.21
  🎯 Fumadores:
    - R² promedio: 0.590
    - MAE promedio: $1718.42
